## <b>■ 복습</b>
    1장. 강화학습이란
        머신러닝의 종류 3가지중 하나
        보상을 통해 학습하는 머신러닝의 종류 
        
    2장. 강화학습을 구현하기 위해서 수학적으로 알아야 할 내용
        MDP
        벨만 방정식
        
    3장. 순차적 프로그래밍 구현하는 방법 중 하나 -> 다이나믹 프로그래밍
        다이나믹 프로그래밍 : 환경을 전부 계산해서 에이전트가 수행해야할 길을 마련해 놓는 것
        다이나믹 프로그래밍의 한계
            1. 환경이 커지면 계산이 오래 걸림
            2. 차원의 저주 (차원이 커지는 경우)
    
    NCS 강화학습 시험 제출
    10월 12일에 GYM의 다른 게임으로 강화학습 시켜서 제출(구현결과, 코드)
    
    4장. 강화학습
        SARSA : 시간차 학습 방법으로 바로바로 환경을 학습하는 방법
            벨만 기대방정식 (On policy)
        QLearning : 행동과 학습을 분리해서 제일 큰 Q함수 값으로 매번 학습하는 방법
            벨만 최적방정식 (Off policy)
            
    5장. SARSA + NN : 얼음판 게임으로 구현
        강화학습 시키는 코드는 하나씩 주석을 달면서 이해
    
    DeepSARSA 코드 구성
        1. 신경망 구현 클래스 (class DeepSARSA) : keras를 상속받아 인공신경망을 3층으로 구성하는 코드
        2. DeepSARSA Agent구현하는 클래스 (class DeepSARSAgent) 
            1. 학습하는 코드
            2. 행동하는 코드
        3. DeepSARSAgent를 실행하는 코드
                      보상, 다음상태
            환경 ------------------------> Agent
                <------------------------ 
                          행동
        
        

In [ ]:
import copy
import pylab
import random
import numpy as np
from environment import Env
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 딥살사 인공신경망
class DeepSARSA(tf.keras.Model): # 부모 클래스인 tf.keras.Model을 상속받은 자식 클래스 생성
    """인공신경망 구현 코드
    """
    def __init__(self, action_size): # 로봇의 행동 규칙을 정해줌
        super(DeepSARSA, self).__init__() # 부모의 유산을 상속받기 위해 작성해야하는 코드
        self.fc1 = Dense(30, activation='relu')
        self.fc2 = Dense(30, activation='relu')
        self.fc_out = Dense(action_size)

    def call(self, x): # 신경망의 순전파 구현 함수
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.fc_out(x)

        return q # 5가지 행동에 대한 Q함수 값

# 그리드월드 예제에서의 딥살사 에이전트
class DeepSARSAgent: 
    """에이전트의 학습과 행동에 대한 메소드를 구현한 클래스
    
    에이전트를 실행하는 실행문
        아래의 상황을 반복하면서 에이전트가 학습하는 코드 구현
                   다음상태, 보상
        환경    ------------------------>   에이전트
               <------------------------
                      
    """
    def __init__(self, state_size, action_size):
        # 상태의 크기와 행동의 크기 정의
        self.state_size = state_size # 입력층의 뉴런의 개수 15개
        self.action_size = action_size # 출력층의 뉴런의 개수 5개

        # 딥살사 하이퍼 파라메터
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1. #무조건 랜덤을 위해 1로 지정
        self.epsilon_decay = .9999 # 랜덤수를 줄이기 위해 깎음
        self.epsilon_min = 0.01 # 0.01까지 깎음
        self.model = DeepSARSA(self.action_size)
        self.optimizer = Adam(lr=self.learning_rate)

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_values = self.model(state)
            return np.argmax(q_values[0])

    # <s, a, r, s', a'>의 샘플로부터 모델 업데이트
    def train_model(self, state, action, reward, next_state, next_action, done):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # 학습 파라메터
        model_params = self.model.trainable_variables # 학습되어져야할 bias와 가중치
        with tf.GradientTape() as tape: # 기울기를 볼 수 있도록 설정
            tape.watch(model_params) # tape에 기록
            predict = self.model(state)[0] # Q함수값 5개 모델 출력
            one_hot_action = tf.one_hot([action], self.action_size) # 행동을 출력하기 위한 OneHot값 출력
            predict = tf.reduce_sum(one_hot_action * predict, axis=1)
            # done = True 일 경우 에피소드가 끝나서 다음 상태가 없음
            
            next_q = self.model(next_state)[0][next_action]
            target = reward + (1 - done) * self.discount_factor * next_q
            
            # MSE 오류 함수 계산
            loss = tf.reduce_mean(tf.square(target - predict))
            
        # 오류함수를 줄이는 방향으로 모델 업데이트
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params)) # 기울기를 가지고 가중치와 bias를 업데이트 한다
        

if __name__ == "__main__": # 최종적으로 실행시키는 스크립트
    # 환경과 에이전트 생성
    env = Env(render_speed=0.01)
    state_size = 15
    action_space = [0, 1, 2, 3, 4]
    action_size = len(action_space)
    agent = DeepSARSAgent(state_size, action_size)
    scores, episodes = [], []
    EPISODES = 1000
    for e in range(EPISODES):
        done = False
        score = 0
        # env 초기화
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        while not done:
            # 현재 상태에 대한 행동 선택
            action = agent.get_action(state)
            # 선택한 행동으로 환경에서 한 타임스텝 진행 후 샘플 수집
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            next_action = agent.get_action(next_state)
            # 샘플로 모델 학습
            agent.train_model(state, action, reward, next_state,
                              next_action, done)
            score += reward
            state = next_state
            if done:
                # 에피소드마다 학습 결과 출력
                print("episode: {:3d} | score: {:3d} | epsilon: {:.3f}".format(
                      e, score, agent.epsilon))
                scores.append(score)
                episodes.append(e)
                pylab.plot(episodes, scores, 'b')
                pylab.xlabel("episode")
                pylab.ylabel("score")
                pylab.savefig("./save_graph/graph.png")
        # 100 에피소드마다 모델 저장
        if e % 100 == 0:
            agent.model.save_weights('save_model/model', save_format='tf')

#### 문제1.  action_space = [0, 1, 2, 3, 4 ] 에서 각각 숫자들의 방향은 어떻게 되는가 ?
    0 : 상
    1 : 하
    2 : 좌
    3 : 우
    4 : 제자리

#### 문제2.  딥살사 인공신경망 모델에 입력되는 state_size  는 입력층의 뉴런의 갯수입니다. 이 뉴런의 갯수를 15개로 지정할 것인데 15개의 입력정보는 각각 무엇입니까 ? 
    에이전트가 장애물을 피하고 도착지점에 가기에 충분한 정보를 에이전트에게 줘야합니다. 
    그래서 사람이 어떤 물체를 피할 때를 생각해보면 정확하지는 않지만 물체의 속도정보가 필요한 것을 알 수 있습니다. 
    물체가 내 쪽으로 오고 있는지 아니면 멀어지고 있는지 등의 정보를 알아야 사람은 자신의 행동을 정할 수 있습니다. 

    상대위치를 적용하는 것은 사람이 물체를 회피할 때 상대적인 거리와 방향을 보기 때문입니다. 


    1. 에이전트에 대한 도착지점의 상대위치 x, y
    2. 도착지점의 라벨
    3. 에이전트에 대한 장애물의 상대위치 x, y
    4. 장애물의 라벨
    5. 장애물의 속도 

    장애물이 총 3개이므로 3번,4번,5번은 3배 해주면 12개이고 여기에 1번과 2번을 더하면 총 15개의 원소를 가집니다. 

    [[ 0  1 -1 -1  1  2 -1 -1  2  3 -1 -1  4  4  1]]

    환경에서 state 를 만드는 함수의 코드는 아래와 같습니다. 

In [ ]:
def get_state(self):

    location = self.coords_to_state(self.canvas.coords(self.rectangle))
    agent_x = location[0]
    agent_y = location[1]

    states = list()

        for reward in self.rewards:
            reward_location = reward['state']
            states.append(reward_location[0] - agent_x)   # 에이전트의 상대위치 x
            states.append(reward_location[1] - agent_y)  # 에이전트의 상대위치 y
            if reward['reward'] < 0:
                states.append(-1)   # 장애물의 라벨
                states.append(reward['direction'])  # 장애물의 속도
            else:
                states.append(1)   # 도착지점의 라벨

        return states

#### 문제3. action_size 로 출력되는 값은 무엇인가요 ? 
    방향에 대한 Q함수 값

In [ ]:
if __name__ == "__main__": 
    # 환경과 에이전트 생성 
    env = Env(render_speed=0.01) 
    state_size = 15 
    action_space = [0, 1, 2, 3, 4] 
    action_size = len(action_space) 
    agent = DeepSARSAgent(state_size, action_size ) 

#### 문제4.  환경이 에이전트에게 주는 다음상태, 보상 과 에이전트가 환경에게 주는 행동을 구현 실행 코드 부분을 한줄 한줄 주석을 다면서 설명하세요 ~

In [ ]:
if __name__ == "__main__": # 최종적으로 실행시키는 스크립트
    # 환경과 에이전트 생성
    env = Env(render_speed=0.01)
    state_size = 15
    action_space = [0, 1, 2, 3, 4]
    action_size = len(action_space)
    agent = DeepSARSAgent(state_size, action_size)
    
    scores, episodes = [], [] # 강화학습이 잘되었는지 총 점수를 담기위한 리스트 선언
    
    EPISODES = 1000 # 게임을 1000회 수행
    
    for e in range(EPISODES):
        done = False # 게임종료 여부를 알기위한 변수 생성
        score = 0 # 게임 점수를 담을 변수 생성
        
        # env 초기화
        state = env.reset() # 게임 환경 초기화
        state = np.reshape(state, [1, state_size]) # 신경망에 입력하기 위해 2차원으로 데이터구성
        
        while not done: # 무한루프
            # 현재 상태에 대한 행동 선택
            action = agent.get_action(state) # 에이전트가 행동을 환경에게 준다
            
            # 선택한 행동으로 환경에서 한 타임스텝 진행 후 샘플 수집
            next_state, reward, done = env.step(action) # 환경이 에이전트에게 다음 상태와 보상, 게임종료여부를 제공
            next_state = np.reshape(next_state, [1, state_size])
            next_action = agent.get_action(next_state) # 환경이 제공한 다음상태를 신경망에 넣고 다음 행동을 출력
            
            # 샘플로 모델 학습
            agent.train_model(state, action, reward, next_state,
                              next_action, done)
            score += reward
            state = next_state
            if done:
                # 에피소드마다 학습 결과 출력
                print("episode: {:3d} | score: {:3d} | epsilon: {:.3f}".format(
                      e, score, agent.epsilon))
                scores.append(score)
                episodes.append(e)
                pylab.plot(episodes, scores, 'b')
                pylab.xlabel("episode")
                pylab.ylabel("score")
                pylab.savefig("./save_graph/graph.png")
        # 100 에피소드마다 모델 저장
        if e % 100 == 0:
            agent.model.save_weights('save_model/model', save_format='tf')

#### 문제5. 훈련할 때 에니메이션을 테스트 때 처럼 천천히 돌게 하시오 !

In [ ]:
import random
import numpy as np
from environment import Env
import tensorflow as tf
from tensorflow.keras.layers import Dense


# 딥살사 인공신경망
class DeepSARSA(tf.keras.Model):
    def __init__(self, action_size):
        super(DeepSARSA, self).__init__()
        self.fc1 = Dense(30, activation='relu')
        self.fc2 = Dense(30, activation='relu')
        self.fc_out = Dense(action_size)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.fc_out(x)
        return q


# 그리드월드 예제에서의 딥살사 에이전트
class DeepSARSAgent:
    def __init__(self, state_size, action_size):
        # 상태의 크기와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        self.epsilon = 0.01
        self.model = DeepSARSA(self.action_size)
        self.model.load_weights('save_model/trained/model')

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_values = self.model(state)
            return np.argmax(q_values[0])


if __name__ == "__main__":
    # 환경과 에이전트 생성
    env = Env(render_speed=0.07)
    state_size = 15
    action_space = [0, 1, 2, 3, 4]
    action_size = len(action_space)
    agent = DeepSARSAgent(state_size, action_size)

    scores, episodes = [], []

    EPISODES = 10
    for e in range(EPISODES):
        score = 0
        done = False
        # env 초기화
        state = env.reset()
        state = np.reshape(state, [1, state_size])

        while not done:
            # 현재 상태에 대한 행동 선택
            action = agent.get_action(state)

            # 선택한 행동으로 환경에서 한 타임스텝 진행 후 샘플 수집
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            state = next_state
            score += reward

            if done:
                # 에피소드마다 학습 결과 출력
                print("episode: {:3d} | score: {:3d}".format(e, score))

### DQN 을 적용전 카트폴 코드 이해하기_문제

In [1]:
import gym
import numpy as np
import random

env = gym.make('CartPole-v1')
goal_steps = 500

for _ in range(30): # 30번 수행
    obs = env.reset() # 게임 초기화
    for i in range(goal_steps): # 환경과 에이전트가 500번 서로 데이터를 주고 받는다
        obs, reward, done, info = env.step(random.randrange(0, 2))
        print(obs) # 현재 상황 출력 [카트 위치, 카트 속도, 막대의 각도, 끝에서의 막대 각도]
        print(reward) # 보상. 한 번이라도 버티면 1.0을 보상으로 준다
        print(done) #  done이 True이면 종료
        print(info) # 빈 dict

        if done:
            print(done)
            break

        env.render()

[ 0.03709713 -0.1457649  -0.0283755   0.31914646]
1.0
False
{}
[ 0.03418183 -0.34047147 -0.02199257  0.60274748]
1.0
False
{}
[ 0.0273724  -0.53527902 -0.00993762  0.88842284]
1.0
False
{}
[ 0.01666682 -0.34002362  0.00783084  0.59263259]
1.0
False
{}
[ 0.00986635 -0.14501217  0.01968349  0.30242661]
1.0
False
{}
[ 0.0069661  -0.34040904  0.02573202  0.60125172]
1.0
False
{}
[ 1.57923383e-04 -5.35881320e-01  3.77570538e-02  9.01927453e-01]
1.0
False
{}
[-0.0105597  -0.34129068  0.0557956   0.62134745]
1.0
False
{}
[-0.01738552 -0.14699046  0.06822255  0.34674608]
1.0
False
{}
[-0.02032533  0.04709811  0.07515747  0.07633235]
1.0
False
{}
[-0.01938336  0.24106665  0.07668412 -0.19172361]
1.0
False
{}
[-0.01456203  0.43501261  0.07284965 -0.45926455]
1.0
False
{}
[-0.00586178  0.23894056  0.06366436 -0.14453771]
1.0
False
{}
[-0.00108297  0.43309573  0.0607736  -0.41647578]
1.0
False
{}
[ 0.00757895  0.2371675   0.05244409 -0.10526956]
1.0
False
{}
[ 0.0123223   0.4315002   0.0503387  -0

[ 0.05973022 -0.35438699  0.06535259  0.80107479]
1.0
False
{}
[ 0.05264248 -0.55034149  0.08137409  1.11357907]
1.0
False
{}
[ 0.04163565 -0.35637685  0.10364567  0.84749245]
1.0
False
{}
[ 0.03450812 -0.16280981  0.12059552  0.58911734]
1.0
False
{}
[ 0.03125192 -0.35939564  0.13237786  0.91722364]
1.0
False
{}
[ 0.02406401 -0.55603498  0.15072234  1.24840752]
1.0
False
{}
[ 0.01294331 -0.75273312  0.17569049  1.58425453]
1.0
False
{}
[-0.00211136 -0.56008196  0.20737558  1.35111396]
1.0
False
{}
[-0.01331299 -0.75711559  0.23439786  1.70086449]
1.0
True
{}
True
[-0.04410175 -0.19166613 -0.01322026  0.28743286]
1.0
False
{}
[-0.04793507  0.00364183 -0.0074716  -0.00939013]
1.0
False
{}
[-0.04786224  0.19887014 -0.00765941 -0.30442104]
1.0
False
{}
[-0.04388483  0.00385817 -0.01374783 -0.01416351]
1.0
False
{}
[-0.04380767 -0.19106395 -0.0140311   0.2741503 ]
1.0
False
{}
[-0.04762895  0.00425536 -0.00854809 -0.02292483]
1.0
False
{}
[-0.04754384 -0.19074297 -0.00900659  0.26704887]
1

[-0.01118442  0.38526019  0.1448771  -0.0951858 ]
1.0
False
{}
[-0.00347922  0.57804061  0.14297339 -0.33888196]
1.0
False
{}
[ 0.0080816   0.38120442  0.13619575 -0.00474911]
1.0
False
{}
[0.01570568 0.18441873 0.13610077 0.32761385]
1.0
False
{}
[ 0.01939406 -0.01235183  0.14265304  0.65993046]
1.0
False
{}
[0.01914702 0.18052706 0.15585165 0.41534883]
1.0
False
{}
[ 0.02275756 -0.01642068  0.16415863  0.75282495]
1.0
False
{}
[ 0.02242915 -0.21337959  0.17921513  1.09233767]
1.0
False
{}
[ 0.01816156 -0.41035158  0.20106188  1.43547124]
1.0
False
{}
[ 0.00995453 -0.60730372  0.22977131  1.78366277]
1.0
True
{}
True
[ 0.03017333  0.21455101  0.0374956  -0.30034125]
1.0
False
{}
[0.03446435 0.01891524 0.03148878 0.00392727]
1.0
False
{}
[ 0.03484266  0.21357177  0.03156732 -0.27865662]
1.0
False
{}
[0.03911409 0.01801405 0.02599419 0.02381308]
1.0
False
{}
[ 0.03947437  0.21275376  0.02647045 -0.26055637]
1.0
False
{}
[ 0.04372945  0.40748803  0.02125932 -0.54477405]
1.0
False
{}
[ 0.

[ 0.10224789 -0.21554592 -0.08839714  0.21660319]
1.0
False
{}
[ 0.09793697 -0.40930027 -0.08406507  0.48014502]
1.0
False
{}
[ 0.08975097 -0.21309848 -0.07446217  0.16219559]
1.0
False
{}
[ 0.085489   -0.40707983 -0.07121826  0.43048968]
1.0
False
{}
[ 0.0773474  -0.21102546 -0.06260847  0.11623313]
1.0
False
{}
[ 0.07312689 -0.01506491 -0.06028381 -0.19552665]
1.0
False
{}
[ 0.07282559 -0.20927506 -0.06419434  0.07754669]
1.0
False
{}
[ 0.06864009 -0.40342082 -0.06264341  0.34930531]
1.0
False
{}
[ 0.06057167 -0.59759849 -0.0556573   0.62159539]
1.0
False
{}
[ 0.0486197  -0.79190085 -0.04322539  0.89624283]
1.0
False
{}
[ 0.03278169 -0.59622036 -0.02530053  0.59029195]
1.0
False
{}
[ 0.02085728 -0.7909791  -0.0134947   0.874899  ]
1.0
False
{}
[ 0.0050377  -0.59567631  0.00400328  0.57800418]
1.0
False
{}
[-0.00687583 -0.79085414  0.01556337  0.87194554]
1.0
False
{}
[-0.02269291 -0.98618426  0.03300228  1.16948067]
1.0
False
{}
[-0.0424166  -1.18171956  0.05639189  1.47232482]
1.0
F

[-0.09637516  0.40730336  0.14543113 -0.24950472]
1.0
False
{}
[-0.08822909  0.21043633  0.14044104  0.08528469]
1.0
False
{}
[-0.08402037  0.40329501  0.14214673 -0.16000104]
1.0
False
{}
[-0.07595446  0.59612608  0.13894671 -0.4046798 ]
1.0
False
{}
[-0.06403194  0.39933532  0.13085312 -0.07161952]
1.0
False
{}
[-0.05604524  0.59236219  0.12942072 -0.3203234 ]
1.0
False
{}
[-0.04419799  0.39565747  0.12301426  0.01020963]
1.0
False
{}
[-0.03628484  0.19900571  0.12321845  0.33903324]
1.0
False
{}
[-0.03230473  0.39217863  0.12999911  0.08760539]
1.0
False
{}
[-0.02446116  0.58522074  0.13175122 -0.16140227]
1.0
False
{}
[-0.01275674  0.77823458  0.12852318 -0.40979096]
1.0
False
{}
[ 0.00280795  0.97132245  0.12032736 -0.65935209]
1.0
False
{}
[ 0.0222344   1.1645825   0.10714032 -0.91185514]
1.0
False
{}
[ 0.04552605  0.96818681  0.08890321 -0.58751179]
1.0
False
{}
[ 0.06488978  0.77193966  0.07715298 -0.26820049]
1.0
False
{}
[ 0.08032858  0.96588061  0.07178897 -0.53558603]
1.0
F

[-0.02470959 -0.01736537  0.02932597  0.17797741]
1.0
False
{}
[-0.02505689  0.17732491  0.03288552 -0.10531183]
1.0
False
{}
[-0.02151039 -0.0182525   0.03077928  0.19756227]
1.0
False
{}
[-0.02187544 -0.21380087  0.03473053  0.49979357]
1.0
False
{}
[-0.02615146 -0.40939479  0.0447264   0.80321624]
1.0
False
{}
[-0.03433936 -0.21491373  0.06079072  0.52493157]
1.0
False
{}
[-0.03863763 -0.02069758  0.07128935  0.25200694]
1.0
False
{}
[-0.03905158 -0.21676127  0.07632949  0.56629679]
1.0
False
{}
[-0.04338681 -0.02278842  0.08765543  0.29860378]
1.0
False
{}
[-0.04384258  0.17098181  0.0936275   0.03480205]
1.0
False
{}
[-0.04042294  0.36464509  0.09432355 -0.22693426]
1.0
False
{}
[-0.03313004  0.55830135  0.08978486 -0.48843738]
1.0
False
{}
[-0.02196401  0.36203499  0.08001611 -0.16886236]
1.0
False
{}
[-0.01472331  0.16586431  0.07663887  0.1479518 ]
1.0
False
{}
[-0.01140603  0.35980989  0.0795979  -0.11960286]
1.0
False
{}
[-0.00420983  0.55370652  0.07720584 -0.38614984]
1.0
F

    obs:  현재 상황을 의미합니다. 이 프로젝트에서는 [카트 위치, 카트 속도, 막대의 각도, 끝에서의 막대의 속도]를 의미합니다. 

    reward: 보상을 의미합니다. 이 프로젝트에서는 오래 버티는게 목표이며, 보상은 밑에서 설명할, 막대가 떨어지지 않는 이상 한 턴을 버텼으므로 1이 주어집니다. 

    done: 막대가 떨어졌는지를 의미합니다. 이 프로젝트에서 막대가 떨어졌다고 말하는 조건은, 막대의 각도가 12도 이상 기울었거나,  
        카트가 2.4칸 이상 움직인 경우(화면에서 나간 경우) 떨어졌다고 판단을 합니다. 

#### 문제1.  다음과 같이 출력되게하시오 ! 

In [ ]:
import gym
import numpy as np
import random

env = gym.make('CartPole-v1')
goal_steps = 500

for _ in range(30): # 30번 수행
    obs = env.reset() # 게임 초기화
    for i in range(goal_steps): # 환경으로부터
        obs, reward, done, info = env.step(random.randrange(0, 2))

        if done:
            # print(done)
            print(f"{_+1}번째에서는 {i}회 버텼습니다")
            break

        env.render()

#### 문제2. 아래와 같이 보상도 같이 출력되게하시오 ! 

In [ ]:
import gym
import numpy as np
import random

env = gym.make('CartPole-v1')
goal_steps = 500

for _ in range(30): # 30번 수행
    obs = env.reset() # 게임 초기화
    for i in range(goal_steps): # 환경으로부터
        obs, reward, done, info = env.step(random.randrange(0, 2))
        print(reward)
        if done:
            # print(done)
            print(f"{_+1}번째에서는 {i}회 버텼습니다")
            break

        env.render()

### (점심시간 문제) cartpole 외에 다른 게임을 실행하고 다른 게임에서 출력하는 obs,reward,done,info값이 무엇인지 확인해보시오

In [ ]:
import gym
import numpy as np
import random

env = gym.make('MsPacman-v0')
goal_steps = 500

for _ in range(30): # 30번 수행
    obs = env.reset() # 게임 초기화
    for i in range(goal_steps): # 환경으로부터
        obs, reward, done, info = env.step(random.randrange(0, 2))
        print(reward)
        if done:
            # print(done)
            print(f"{_+1}번째에서는 {i}회 버텼습니다")
            break

        env.render()